# Player Dataprep

In [ ]:
from nba_api.stats.endpoints import leaguegamelog
import pandas as pd

In [16]:
from nba_api.stats.endpoints import leaguegamelog
import pandas as pd

YEAR = 2023

COUNTER =  0
DIRECTION = "ASC"
LEAGUE = "00"
PLAYER_OR_TEAM = "P"
SEASON_TYPE = "Regular Season"
SORTER = "DATE"

def fetch_season_game_logs(season):
    # Create LeagueGameLog object instance for the season
    gamelog = leaguegamelog.LeagueGameLog(
        counter=COUNTER,
        direction=DIRECTION,
        league_id = LEAGUE,
        player_or_team_abbreviation=PLAYER_OR_TEAM,
        season=season,
        season_type_all_star=SEASON_TYPE,
        sorter=SORTER
    )
    
    # Execute request and fetch data
    data = gamelog.get_data_frames()[0] 
    
    # Filter columns and add Home/Away column
    columns_to_keep = [
       "GAME_ID", "TEAM_ID", "TEAM_NAME",  "PLAYER_ID", "PLAYER_NAME", "MATCHUP", "WL", "GAME_DATE",
        "PTS", "OREB", "DREB", "AST", "STL", "BLK", "TOV", "FGM", "FGA", "FG_PCT",
        "FG3M", "FG3A", "FG3_PCT", "FTM", "FTA", "FT_PCT", "MIN"
    ]
    filtered_data = data[columns_to_keep].copy()
    filtered_data['HOME/AWAY'] = filtered_data['MATCHUP'].apply(lambda x: 0 if '@' in x else 1)
    
    return filtered_data

def fetch_multiple_seasons(start_year, end_year):
    all_seasons_data = []  # List to store data for all seasons
    
    # Loop through each season from start_year to end_year (inclusive)
    for year in range(start_year, end_year + 1):
        season = f"{year}-{str(year + 1)[-2:]}"  # Format season string (e.g., '2010-11')
        season_data = fetch_season_game_logs(season)
        all_seasons_data.append(season_data)
    
    # Concatenate all seasons' data into a single DataFrame
    combined_data = pd.concat(all_seasons_data, ignore_index=True)
    combined_data = combined_data.sort_values(by='GAME_ID', ascending=True)
    combined_data = combined_data.reset_index(drop=True)
    
    return combined_data

# Example usage for fetching player game logs for the 2023 season
player_game_logs = fetch_multiple_seasons(2023, 2023)
player_game_logs

,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,GAME_ID,MATCHUP,WL,GAME_DATE,PTS,OREB,...,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,MIN,HOME/AWAY
0,1627747,Caris LeVert,1610612739,Cleveland Cavaliers,0022300001,CLE @ IND,L,2023-11-03,13,0,...,12,0.417,0,4,0.0,3,4,0.75,30,0
1,1629636,Darius Garland,1610612739,Cleveland Cavaliers,0022300001,CLE @ IND,L,2023-11-03,14,0,...,11,0.455,0,2,0.0,4,4,1.00,32,0
2,1629614,Andrew Nembhard,1610612754,Indiana Pacers,0022300001,IND vs. CLE,W,2023-11-03,6,0,...,5,0.400,0,0,NaN,2,2,1.00,18,1
3,1630171,Isaac Okoro,1610612739,Cleveland Cavaliers,0022300001,CLE @ IND,L,2023-11-03,5,0,...,3,0.667,1,1,1.0,0,0,NaN,23,0
4,1630174,Aaron Nesmith,1610612754,Indiana Pacers,0022300001,IND vs. CLE,W,2023-11-03,13,1,...,8,0.625,2,4,0.5,1,4,0.25,27,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26396,1630230,Naji Marshall,1610612740,New Orleans Pelicans,0022301230,NOP @ LAL,L,2023-12-07,5,2,...,2,0.500,1,1,1.0,2,2,1.00,12,0
26397,2544,LeBron James,1610612747,Los Angeles Lakers,0022301230,LAL vs. NOP,W,2023-12-07,30,0,...,12,0.750,4,4,1.0,8,8,1.00,23,1
26398,203076,Anthony Davis,1610612747,Los Angeles Lakers,0022301230,LAL vs. NOP,W,2023-12-07,16,5,...,10,0.500,0,2,0.0,6,8,0.75,31,1
26399,1629629,Cam Reddish,1610612747,Los Angeles Lakers,0022301230,LAL vs. NOP,W,2023-12-07,9,2,...,6,0.500,1,2,0.5,2,2,1.00,23,1
